In [ ]:
# REQUIREMENTS
# pip install devtools
# pip install specklepy
# pip install requests-toolbelt
# pip install websockets

In [1]:
import ujson as json
from devtools import debug
from specklepy.api import operations
from specklepy.api.client import SpeckleClient
from specklepy.api.credentials import get_default_account
from specklepy.api.wrapper import StreamWrapper

from specklepy.transports.server import ServerTransport
from specklepy.transports.memory import MemoryTransport
from specklepy.transports.sqlite import SQLiteTransport

In [22]:
from specklepy.api.client import SpeckleClient
from specklepy.api.credentials import get_default_account

# initialise the client
client = SpeckleClient(host="https://speckle.xyz") # or whatever your host is
# client = SpeckleClient(host="localhost:3000", use_ssl=False) or use local server

# authenticate the client with a token
# account = get_default_account()
# client.authenticate_with_token("8349dc1b415baafb37d91c244867dc5c5a6f0412d8") # python token
client.authenticate_with_token("0ca1b861abd574e214446e26335e8772f1a4c09ad4") # python2 token

# if you're in an environment without accounts, you can construct an Account object yourself
# or authenticate with just a token
# client.authenticate_with_token("YOUR_TOKEN")

In [17]:
from specklepy.api.credentials import get_default_account, get_local_accounts
all_accounts = get_local_accounts() # get back a list

In [18]:
all_accounts

[Account(email: orzan.daniel@yahoo.it, server: https://speckle.xyz, isDefault: True)]

In [23]:
# create a new stream. this returns the stream id
new_stream_id = client.stream.create(name="a shiny new stream")

# use that stream id to get the stream from the server
new_stream = client.stream.get(id=new_stream_id)

In [24]:
from specklepy.objects import Base
from specklepy.objects.geometry import Point

class Block(Base):
    length: float
    width: float
    height: float
    origin: Point = None

    def __init__(self, length=1.0, width=1.0, height=1.0, origin=Point(), **kwargs) -> None:
        super().__init__(**kwargs)
        # mark the origin as a detachable attribute
        self.add_detachable_attrs({"origin"})

        self.length = length
        self.width = width
        self.height = height
        self.origin = origin

In [25]:
from specklepy.transports.server import ServerTransport
from specklepy.api import operations

# here's the data you want to send
block = Block(length=2, height=4)

# next create a server transport - this is the vehicle through which you will send and receive
transport = ServerTransport(client=client, stream_id=new_stream_id)

# this serialises the block and sends it to the transport
hash = operations.send(base=block, transports=[transport])

# you can now create a commit on your stream with this object
commid_id = client.commit.create(
    stream_id=new_stream_id, 
    object_id=hash, 
    message="this is a block I made in speckle-py",
    )

In [26]:
# this receives the object back from the transport.
# the received data will be deserialised back into a `Block` 
received_base = operations.receive(obj_id=hash, remote_transport=transport)

In [6]:
from specklepy.api.wrapper import StreamWrapper

# provide any stream, branch, commit, object, or globals url
wrapper = StreamWrapper("https://speckle.xyz/streams/c4a1a7c48b/commits/50385f68d2")

# get an authenticated SpeckleClient if you have a local account for the server
client = wrapper.get_client()

# get an authenticated ServerTransport if you have a local account for the server
transport = wrapper.get_transport()

In [27]:
# get a list of your most recent streams
stream_list = client.stream.list()
stream_list

[Stream( id: c4a1a7c48b, name: OUH - Rooms, description: Rooms Management, isPublic: True)]